In [18]:
#Import dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import time

In [19]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## Mars News

In [26]:
#URL information of the page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
time.sleep(3)
html=browser.html
soup=bs(html, 'html.parser')

In [27]:
#Retrieve latest news title and paragraph text (use variables for reference later)
results=soup.find('li', class_='slide')

news_title = results.find('div', class_='content_title').text
news_p = results.find('div', class_='article_teaser_body').text

In [28]:
news_title

"Sensors Prepare to Collect Data as Perseverance Enters Mars' Atmosphere"

In [29]:
news_p

'Technology will collect critical data about the harsh entry environment during Perseverance’s entry next Thursday.'

## Featured Image

In [30]:
image_url='https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(image_url)

In [31]:
html=browser.html
soup=bs(html, 'html.parser')

In [32]:
image_tag=soup.find('img', class_='headerimage')
partial_url=image_tag.get('src')
featured_image_url=(f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{partial_url}')
featured_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg'

## Mars Facts

In [33]:
#Scrape Mars facts
facts_url='https://space-facts.com/mars/'
mars_facts_df=pd.read_html(facts_url)[0]
print(mars_facts_df)

                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


In [34]:
mars_facts_df.columns=['Characteristics', 'Mars']
mars_facts_df.set_index('Characteristics', inplace=True)
mars_facts_df

,Mars
Characteristics,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [35]:
mars_table_html=mars_facts_df.to_html(index=True, header=True, border=0, justify="left")
print(mars_table_html)

<table border="0" class="dataframe">
  <thead>
    <tr style="text-align: left;">
      <th></th>
      <th>Mars</th>
    </tr>
    <tr>
      <th>Characteristics</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [36]:
mars_table_html=mars_facts_df.to_html('mars_table_html', index=True, header=True, border=0, justify="left")

## Mars Hemispheres

In [40]:
#Scrape Mars hemisphere title and image
usgs_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_url)

In [41]:
html=browser.html
soup=bs(html, 'html.parser')

In [42]:
hem_list=[]
count=0
mars_hemi_url='https://astrogeology.usgs.gov/'
hemisphere_locate=soup.find_all('div', class_='item')

for results in hemisphere_locate:
    mars_title=results.find('h3').text
    browser.links.find_by_partial_text(mars_title).click()
    time.sleep(1)

    html=browser.html
    soup=bs(html, 'html.parser')
    partial_image=soup.find('img', class_='wide-image')['src']

    img_url=mars_hemi_url+partial_image

    #Create list of dictionaries
    hem_list.append({
        'title':mars_title,
        'img_url':img_url
    })

    count=count+1
    
    if len(hemisphere_locate)>count:
        browser.back()
        time.sleep(1)
    else:
        break

hem_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [43]:
browser.quit()